## 01 Introduction and Setup
## 02 V8 The Javascript Engine
## 03 The Node Core

## 04 Modules Exports and Require
### 027 exports vs module.exports
require() 实际上是类似下面的 wrapper 函数。其中第一个参数其实就是 module.exports。
```
fn(module.exports, require, module, __filename, __dirname);
```

In [1]:
(function (exports, require, module, __filename, __dirname) {

    var greet = function() {
        console.log('Hello!');
    };

    module.exports = greet;
});

function (exports, require, module, __filename, __dirname) {

    var greet = function() {
        console.log('Hello!');
    };

    module.exports = greet;
}

In [2]:
// 模拟 exports 换成其它值
(function() {

    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        exports = function() {
            console.log('Hello!');
        };

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

function () {
            console.log('Hello!');
        }
{}


{}

上面例子让 exports 指向了不同的对象。**一般可能期望 exports 改了，module.exports 也跟着修改，但这里不是这样。**

In [3]:
// 再模拟，改变 exports 内部（mutate exports），但 exports 本身引用不变
(function() {

    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        // NOTE: 由于某些原因，函数并不会输出，所以又加了字符串属性来验证
        exports.greet = function() { console.log('Hello!'); };
        exports.greetStr = `function() { console.log('Hello!'); }`;

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

{"greetStr":"function() { console.log('Hello!'); }"}
{"greetStr":"function() { console.log('Hello!'); }"}


{"greetStr":"function() { console.log('Hello!'); }"}

这里容易造成误解，所以简单的方法：***Just use module.exports**

### 028 Requiring Native Core Modules

In [4]:
var util = require('util');

var name = 'Tony';
var greeting = util.format('Hello, %s', name);
//util.log(greeting); // jupyter 里面似乎不工作
console.log(greeting);

Hello, Tony


如果自己也有同名的 module 怎么办？问题不大，自己的使用相对路径：
```
require('./util');
```
但是还是不建议使用相同的名字，造成混淆。

### 029 Modules and ES6

- greet.js
```
export function greet() {
    console.log('Hello');
}
```
- app.js
```
import * as greetr from 'greet';
greetr.greet();
```

## 05 Events and the Event Emitter

### 031 Conceptual Aside Events
- System Events
  - C++ Core (libuv)
- Custom Event
  - Javascript Core (Event Emitter)

### 033 The Node Event Emitter - Part 1
下面是一个模拟的 Event Emitter

In [5]:
(function() {
    var module = {};
    var require = function() {
        return module.exports;
    };

    // ----- begin of emitter.js -----
    function Emitter() {
        this.events = {};
    }

    Emitter.prototype.on = function(type, listener) {
        this.events[type] = this.events[type] || [];
        this.events[type].push(listener);
    };

    Emitter.prototype.emit = function(type) {
        if (this.events[type]) {
            this.events[type].forEach(function(listener) {
                listener();
            });
        }
    }
    
    module.exports = Emitter;
    // ----- end of emitter.js -----

    // ----- begin of app.js -----
    var Emitter = require('./emitter');
    var emtr = new Emitter();

    emtr.on('greet', function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on('greet', function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit('greet');
})();

Hello!
Somewhere, some said hello.
A greeting occurred!


### 034 The Node Event Emitter - Part 2
Node 提供的 Event Emitter

In [6]:
(function() {
    var Emitter = require('events');
    var emtr = new Emitter();

    emtr.on('greet', function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on('greet', function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit('greet');
})();

Hello!
Somewhere, some said hello.
A greeting occurred!


怎么避免使用这些 magic strings?

In [7]:
(function() {
    var module = {};
    var _require = require;
    require = function(name) {
        return (name === './config') ? module.exports : _require(name);
    };

    // ----- begin of config.js -----
    module.exports = {
        events: {
            GREET: 'greet'
        }
    };

    // ----- begin of app.js -----
    var Emitter = require('events');
    var eventConfig = require('./config').events;
    var emtr = new Emitter();

    emtr.on(eventConfig.GREET, function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on(eventConfig.GREET, function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit(eventConfig.GREET);
})();

Hello!
Somewhere, some said hello.
A greeting occurred!
